# Importações

In [1]:
import pandas as pd
from os import listdir
from os.path import isfile, join
import numpy as np
from sklearn.impute import KNNImputer

# Criando variáveis organizacionais

In [2]:
path = '../raw_data/SP' #caminho geral
files = [f for f in listdir(path) if isfile(join(path, f))] #lista de nomes de arquivos de dados

# Tratamentos iniciais dos dados

## tratamentos básicos

In [3]:
#---------------------fazer list com os dataframes---------------------
df_list = []
for file in range(0,88):
    df = pd.read_csv(f'../raw_data/SP/{files[file]}', sep=';', skiprows=8, encoding="ISO-8859-1", decimal=',')
    lat_log_alt = pd.read_csv(f'../raw_data/SP/{files[file]}', sep=';', skiprows=4,
                          nrows=3, encoding="ISO-8859-1", decimal=',', names=['lat_lon_alt','valor'])
    df['Estaçao']=files[file].split('_')[4]
    df['Latitude']=lat_log_alt['valor'][0]
    df['Longitude']=lat_log_alt['valor'][1]
    df['Altitude']=lat_log_alt['valor'][2]
    df_list.append(df)
df_list

#---------------------fundir os dataframes no dataframe vazio---------------------
full_df = pd.concat(df_list)
full_df
df2 = full_df.copy()

#---------------------dropar coluna inútil---------------------
df2.drop(columns=["Unnamed: 19"],inplace=True)

#---------------------mudar nome das colunas---------------------
df2= df2.rename(columns={'Data': 'Data',
                                            'Hora UTC': 'Hora(UTC)',
                                            'PRECIPITAÇÃO TOTAL, HORÁRIO (mm)': 'Chuva',
                                            'PRESSAO ATMOSFERICA AO NIVEL DA ESTACAO, HORARIA (mB)': 'Pres',
                                            'PRESSÃO ATMOSFERICA MAX.NA HORA ANT. (AUT) (mB)': 'Pres.max',
                                            'PRESSÃO ATMOSFERICA MIN. NA HORA ANT. (AUT) (mB)': 'Pres.min',
                                            'RADIACAO GLOBAL (Kj/m²)': 'Radiacao',
                                            'TEMPERATURA DO AR - BULBO SECO, HORARIA (°C)': 'Temp',
                                            'TEMPERATURA DO PONTO DE ORVALHO (°C)': 'Temp.orvalho',
                                            'TEMPERATURA MÁXIMA NA HORA ANT. (AUT) (°C)': 'Temp.max',
                                            'TEMPERATURA MÍNIMA NA HORA ANT. (AUT) (°C)': 'Temp.min',
                                            'TEMPERATURA ORVALHO MAX. NA HORA ANT. (AUT) (°C)': 'Temp.orvalho.max',
                                            'TEMPERATURA ORVALHO MIN. NA HORA ANT. (AUT) (°C)': 'Temp.orvalho.min',
                                            'UMIDADE REL. MAX. NA HORA ANT. (AUT) (%)': 'Umid.max',
                                            'UMIDADE REL. MIN. NA HORA ANT. (AUT) (%)': 'Umid.min',
                                            'UMIDADE RELATIVA DO AR, HORARIA (%)': 'Umid',
                                            'VENTO, DIREÇÃO HORARIA (gr) (° (gr))': 'Dir.vento',
                                            'VENTO, RAJADA MAXIMA (m/s)': 'Rajada.vento',
                                            'VENTO, VELOCIDADE HORARIA (m/s)': 'Vel.vento'})

## Transformação de dados e preenchimento

### formatando data e hora

In [4]:
#---------------------colocando data e hora no formato adequado---------------------
#df2["Data"] = pd.to_datetime(df2["Data"])
df2['datahora'] = df2.apply(lambda row: pd.to_datetime(f'{row["Data"]} {row["Hora(UTC)"]}'), axis=1)
df2['datahora'].dt.hour

0        0
1        1
2        2
3        3
4        4
        ..
8779    19
8780    20
8781    21
8782    22
8783    23
Name: datahora, Length: 771936, dtype: int64

### transformando chuva em variável categórica

In [5]:
#---------------------transformando a chuva em variável categórica---------------------
def classe_chuva(precipitacao):
    mm=precipitacao
    if mm == 0:
        chuva = 'nao chove'
    elif mm >0 and mm <=2.5:
        chuva = 'fraca'
    elif mm >2.5 and mm<=7.5:
        chuva = 'moderada'
    elif mm >7.5 and mm<=50:
        chuva = 'forte'
    else:
        chuva = 'muito forte'
    return chuva
df2['classe_chuva'] = df2['Chuva'].apply(lambda x: classe_chuva(x))

### tratando valores nulos de radiação

In [19]:
#---------------------Tratando valores nulos de radiação---------------------
def tratar_radiacao(hora,radiacao):
    if (df2['datahora'].dt.hour > 22) or (df2['datahora'].dt.hour < 8):
        df2['Radiacao'] = df2['Radiacao'].replace(np.nan, 0)
    return radiacao
df2['Radiacao'] = df2.apply(lambda x: tratar_radiacao(x['datahora'],x['Radiacao']),axis=1)

ValueError: The truth value of a Series is ambiguous. Use a.empty, a.bool(), a.item(), a.any() or a.all().

### imputer da coluna de vento

In [ ]:
#---------------------fazer imputer nas colunas de vento---------------------
imputer = KNNImputer(n_neighbors=2)
imputer.fit_transform(df2[['Rajada.vento', 'Vel.vento', 'Dir.vento']])

# Explorando os dados

In [11]:
df2['datahora'].dt.hour


0        0
1        1
2        2
3        3
4        4
        ..
8779    19
8780    20
8781    21
8782    22
8783    23
Name: datahora, Length: 771936, dtype: int64

In [9]:
df2

,Data,Hora(UTC),Chuva,Pres,Pres.max,Pres.min,Radiacao,Temp,Temp.orvalho,Temp.max,...,Umid,Dir.vento,Rajada.vento,Vel.vento,Estaçao,Latitude,Longitude,Altitude,datahora,classe_chuva
0,2020/01/01,0000 UTC,0.0,923.1,923.1,922.1,NaN,22.0,20.5,22.2,...,92.0,172.0,2.8,0.4,SAO PAULO - INTERLAGOS,-23.724501,-46.677501,771.00,2020-01-01 00:00:00+00:00,nao chove
1,2020/01/01,0100 UTC,0.0,923.7,923.7,923.1,NaN,22.1,20.7,22.3,...,92.0,101.0,1.8,0.9,SAO PAULO - INTERLAGOS,-23.724501,-46.677501,771.00,2020-01-01 01:00:00+00:00,nao chove
2,2020/01/01,0200 UTC,0.0,923.6,923.8,923.6,NaN,23.2,20.1,23.5,...,82.0,7.0,3.3,1.2,SAO PAULO - INTERLAGOS,-23.724501,-46.677501,771.00,2020-01-01 02:00:00+00:00,nao chove
3,2020/01/01,0300 UTC,0.0,923.1,923.6,923.1,NaN,23.3,19.5,23.6,...,79.0,6.0,3.0,1.2,SAO PAULO - INTERLAGOS,-23.724501,-46.677501,771.00,2020-01-01 03:00:00+00:00,nao chove
4,2020/01/01,0400 UTC,0.0,922.6,923.1,922.6,NaN,22.9,19.0,23.6,...,79.0,345.0,3.5,0.6,SAO PAULO - INTERLAGOS,-23.724501,-46.677501,771.00,2020-01-01 04:00:00+00:00,nao chove
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8779,2020/12/31,1900 UTC,0.0,951.5,951.6,951.3,1477.3,27.8,20.6,28.9,...,65.0,350.0,4.1,1.6,BARRETOS,-20.559167,-48.545000,534.36,2020-12-31 19:00:00+00:00,nao chove
8780,2020/12/31,2000 UTC,0.0,951.6,951.7,951.3,746.7,27.2,20.4,28.0,...,67.0,301.0,3.8,1.1,BARRETOS,-20.559167,-48.545000,534.36,2020-12-31 20:00:00+00:00,nao chove
8781,2020/12/31,2100 UTC,0.0,951.4,951.6,951.3,321.1,25.3,22.0,27.2,...,82.0,119.0,2.6,0.6,BARRETOS,-20.559167,-48.545000,534.36,2020-12-31 21:00:00+00:00,nao chove
8782,2020/12/31,2200 UTC,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,BARRETOS,-20.559167,-48.545000,534.36,2020-12-31 22:00:00+00:00,muito forte


In [7]:
#verificando células nulas
df2.isnull().sum().sort_values(ascending=False)/len(df2)

Radiacao            0.616199
Chuva               0.424913
Rajada.vento        0.358421
Dir.vento           0.356820
Vel.vento           0.356813
Temp.orvalho.min    0.335100
Temp.orvalho.max    0.334880
Umid.max            0.334200
Umid.min            0.334160
Temp.orvalho        0.331385
Umid                0.331277
Temp.max            0.320122
Temp.min            0.320120
Pres.min            0.319472
Pres.max            0.319459
Temp                0.317793
Pres                0.317138
datahora            0.000000
Longitude           0.000000
Altitude            0.000000
Data                0.000000
Latitude            0.000000
Estaçao             0.000000
Hora(UTC)           0.000000
classe_chuva        0.000000
dtype: float64